In [ ]:
#| default_exp datasources.sitarweb
%load_ext autoreload
%autoreload 2

import sys,os
from pathlib import Path

In [ ]:
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')

# SITARWEB
> Módulo para encapsular a extração e processamento das bases de dados do SITARWEB

In [ ]:
#| export
import os
from decimal import Decimal, getcontext
from functools import cached_property

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from fastcore.foundation import GetAttr

from extracao.constants import (
	COLUNAS,
	SQL_RADCOM,
	SQL_STEL,
)
from extracao.datasources.base import Base
from extracao.datasources.connectors import SQLServer

In [ ]:
#| export
getcontext().prec = 5
load_dotenv(find_dotenv(), override=True)

True

In [ ]:
#| hide: true
#| eval:false
__file__ = Path.cwd().parent / 'extracao' / 'datasources.py'

In [ ]:
#| export

SQLSERVER_PARAMS = dict(
	driver='{ODBC Driver 17 for SQL Server}',
	server='ANATELBDRO05',
	database='SITARWEB',
	trusted_conn=True,
	mult_results=True,
	encrypt=False,
	username=os.environ.get('USERNAME'),
	password=os.environ.get('PASSWORD'),
	timeout=1000,
)


class Sitarweb(Base, GetAttr):
	def __init__(self, sql_params: dict = SQLSERVER_PARAMS):
		self.default = SQLServer(sql_params)

	@property
	def columns(self):
		return COLUNAS

	@property
	def query(self):
		raise NotImplementedError("Subclasses devem implementar a propriedade 'query'")

	def extraction(self):
		return pd.read_sql_query(self.query, self.connect(), dtype='category')


In [ ]:
#| export

class Radcom(Sitarweb):
	def __init__(self, sql_params: dict = SQLSERVER_PARAMS):
		super().__init__(sql_params)

	@property
	def query(self):
		return SQL_RADCOM

	@property
	def stem(self):
		return 'radcom'

	def _format(
		self,
		df: pd.DataFrame,  # DataFrame com o resultantes do banco de dados
	) -> pd.DataFrame:  # DataFrame formatado
		"""Formata, limpa e padroniza os dados provenientes da query no banco"""
		df['Entidade'] = df['Entidade'].str.strip()
		df['Num_Serviço'] = '231'
		df['Classe_Emissão'] = pd.NA
		df['Largura_Emissão(kHz)'] = '256'
		df['Validade_RF'] = pd.NA
		df['Status'] = 'RADCOM'
		df['Fonte'] = 'SRD'
		df['Multiplicidade'] = '1'
		a = df.Situação.isna()
		df.loc[a, 'Classe'] = df.loc[a, 'Fase'].astype('string')
		df.loc[~a, 'Classe'] = (
			df.loc[~a, 'Fase'].astype('string') + '-' + df.loc[~a, 'Situação'].astype('string')
		)
		df.drop(['Fase', 'Situação'], axis=1, inplace=True)
		df['Log'] = ''
		df['Frequência'] = pd.to_numeric(df['Frequência'], errors='coerce').astype('float')
		discarded = df[df.Frequência.isna()].copy()
		if not discarded.empty:
			log = f"""[("Colunas", "Frequência"),  
            ("Processamento", "Valor Nulo")]"""
			self.append2discarded(self.register_log(discarded, log))
		df.dropna(subset=['Frequência'], inplace=True)
		return df.loc[:, self.columns]


In [ ]:
#| export

class Stel(Sitarweb):
	def __init__(self, sql_params: dict = SQLSERVER_PARAMS):
		super().__init__(sql_params)

	@property
	def query(self):
		return SQL_STEL

	@property
	def stem(self):
		return 'stel'

	def _format(
		self,
		df: pd.DataFrame,  # DataFrame com o resultantes do banco de dados
	) -> pd.DataFrame:  # DataFrame formatado
		"""Formata, limpa e padroniza os dados provenientes da query no banco"""
		df['Status'] = 'L'
		df['Entidade'] = df.Entidade.str.strip()
		df['Fonte'] = 'STEL'
		df['Largura_Emissão'] = df['Largura_Emissão'].astype('string')
		df.loc[:, ['Largura_Emissão(kHz)', '_']] = (
			df.Largura_Emissão.fillna('').apply(self.parse_bw).tolist()
		)
		df.drop(['Largura_Emissão', '_'], axis=1, inplace=True)
		df.loc[:, 'Validade_RF'] = df.Validade_RF.astype('string').str.slice(0, 10)
		df['Frequência'] = df['Frequência'].astype('float')
		df.loc[df.Unidade == 'kHz', 'Frequência'] = df.loc[df.Unidade == 'kHz', 'Frequência'].apply(
			lambda x: float(Decimal(x) / Decimal(1000))
		)
		df.loc[df.Unidade == 'GHz', 'Frequência'] = df.loc[df.Unidade == 'GHz', 'Frequência'].apply(
			lambda x: float(Decimal(x) * Decimal(1000))
		)
		df.drop('Unidade', axis=1, inplace=True)
		df['Multiplicidade'] = 1
		df['Log'] = ''
		return df.loc[:, self.columns]


In [ ]:
#| export
if __name__ == '__main__':
	import time

	start = time.perf_counter()

	data = Stel()

	data.update()

	print('STEL')

	display(data.df)

	data.save()

	print(150 * '=')

	data = Radcom()

	data.update()

	print('STEL')

	display(data.df)

	data.save()

	print(f'Elapsed time: {time.perf_counter() - start} seconds')

C:\Users\rsilva\AppData\Local\Temp\2\ipykernel_12504\3887925753.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(self.query, self.connect(), dtype="string")


STEL


,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade,Log
0,156.6,PETROLEO BRASILEIRO S A PETROBRAS,50411168908,1012361591,Macaé,3302403,RJ,-22.4852305555555,-40.09588333333333,ML,604,F3E,11.0,2033-12-03,L,STEL,1,[]
1,156.475,PETROLEO BRASILEIRO S A PETROBRAS,50411168908,1004693106,Macaé,3302403,RJ,-21.90675,-39.736041666666665,FC,604,F3E,16.0,2033-12-03,L,STEL,1,[]
2,469.7625,COMANDO DA AERONAUTICA,50408035161,695834789,Salvador,2927408,BA,-12.908611111111,-38.3225,FB,035,F2D,12.5,2012-07-28,L,STEL,1,[]
3,4.4830,Oceanpact Servicos Maritimos S.a.,50439754828,1012385849,Arraial do Cabo,3300258,RJ,-23.012461111111,-42.001425,FC,035,P0N,70.0,2024-02-25,L,STEL,1,[]
4,156.6,Cooperativa Unipilot - Cooperativa de Apoio e ...,50439872430,1012402409,Santarém,1506807,PA,-2.419811111111,-54.726486111111,FC,604,G3E,16.0,2041-09-28,L,STEL,1,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35275,7680.0,WKVE-ASSES. EM SERV. DE INF. E TELECOMUNICACOE...,50405354223,695977938,São João do Araguaia,1507508,PA,-5.4755722222221666,-48.856386111111,FX,046,D7W,28000.0,2033-08-14,L,STEL,1,[]
35276,7777.35,VIPREDE TECNOLOGIA LTDA,50407348751,1002925417,Presidente Kennedy,3204302,ES,-21.1008055555555,-41.04147222222217,FX,046,D1D,29700.0,2035-09-03,L,STEL,1,[]
35277,949.125,GENOA FM LTDA,50411450166,1000394597,Siqueira Campos,4126603,PR,-23.689444444444334,-49.8280555555555,TX,251,F3E,180.0,2030-03-16,L,STEL,1,[]
35278,938.0,UNIVERSAL FM STEREO BRODOWSKI LTDA - ME,02022648587,359067050,Ribeirão Preto,3543402,SP,-21.184833333333334,-47.81244444444433,TX,251,D7W,250.0,2017-12-21,L,STEL,1,[]


C:\Users\rsilva\AppData\Local\Temp\2\ipykernel_12504\3887925753.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(self.query, self.connect(), dtype="string")


STEL


,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade,Log
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,1008210959,Turilândia,2112456,MA,-2.228611111111,-45.306666666666665,P-A,231,NaN,256,NaN,RADCOM,SRD,1,[]
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,699491851,Alvorada de Minas,3102407,MG,-18.734166666666667,-43.36472222222217,3,231,NaN,256,NaN,RADCOM,SRD,1,[]
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,699359830,Congonhas do Norte,3118106,MG,-18.812777777777665,-43.673611111111,3,231,NaN,256,NaN,RADCOM,SRD,1,[]
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,690859562,Dom Joaquim,3122603,MG,-18.95,-43.266666666666666,3,231,NaN,256,NaN,RADCOM,SRD,1,[]
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,1008401606,Castanheira,5102850,MT,-11.137222222222167,-58.61333333333334,P-K,231,NaN,256,NaN,RADCOM,SRD,1,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,107.9,ASSOCIAÇÃO CULT COMUNIT MORUMBI,50407431578,697707695,São José dos Campos,3549904,SP,-23.267777777777667,-45.8975,3-B,231,NaN,256,NaN,RADCOM,SRD,1,[]
4997,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50406232911,692270272,São José dos Campos,3549904,SP,-23.5530555555555,-45.8705555555555,3,231,NaN,256,NaN,RADCOM,SRD,1,[]
4998,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50434484237,692270272,São José dos Campos,3549904,SP,-23.5530555555555,-45.8705555555555,3,231,NaN,256,NaN,RADCOM,SRD,1,[]
4999,107.9,ASSOCIACAO COMUNITARIA CULTURAL DE MUSICA E CI...,50406778205,693049723,São José dos Campos,3549904,SP,-23.191944444444335,-45.87527777777767,3,231,NaN,256,NaN,RADCOM,SRD,1,[]


Elapsed time: 11.747743700165302 seconds
